In [3]:
%load_ext dotenv
%dotenv 

# What are we doing?

## Objectives 


* Build a data pipeline that downloads price data from the internet, stores it locally, transforms it into return data, and stores the feature set.
    - Getting the data.
    - Schemas and index in dask.

* Explore the parquet format.
    - Reading and writing parquet files.
    - Read datasets that are stored in distributed files.
    - Discuss dask vs pandas as a small example of big vs small data.
    
* Discuss the use of environment variables for settings.
* Discuss how to use Jupyter notebooks and source code concurrently. 
* Logging and using a standard logger.

## About the Data

+ We will download the prices for a list of stocks.
+ The source is Yahoo Finance and we will use the API provided by the library yfinance.


## Medallion Architecture

+ The architecture that we are thinking about is called Medallion by [DataBricks](https://www.databricks.com/glossary/medallion-architecture). It is an ELT type of thinking, although our data is well-structured.

![Medallion Architecture (DataBicks)](./images/02_medallion_architecture.png)

+ In our case, we would like to optimize the number of times that we download data from the internet. 
+ Ultimately, we will build a pipeline manager class that will help us control the process of obtaining and transforming our data.

![](./images/02_target_pipeline_manager.png)

# Download Data from Yahoo Finance

Yahoo Finance provides information about public stocks in different markets. The library yfinance gives us access to a fair bit of the data in Yahoo Finance. 

These steps are based on the instructions in:

+ [yfinance documentation](https://pypi.org/project/yfinance/)
+ [Tutorial in geeksforgeeks.org](https://www.geeksforgeeks.org/get-financial-data-from-yahoo-finance-with-python/)


+ If required, install: `python -m pip install yfinance`.
+ To download the price history of a stock, first use the following setup:


In [4]:
import pandas as pd
import yfinance as yf
import os
import sys

sys.path.append(os.getenv('SRC_DIR'))

A few things to notice in the code chunk above:

+ Libraries are ordered from high-level to low-level libraries from the package manager (pip in this case, but could be conda, poetry, etc.)
+ The command `sys.path.append("../05_src/)` will add the `../05_src/` directory to the path in the Notebook's kernel. This way, we can use our modules as part of the notebook.
+ Local modules are imported at the end. 
+ The function `get_logger()` is called with `__name__` as recommended by the documentation.

Now, to download the historical price data for a stock, we could use:

In [5]:
stock = yf.Ticker("AAPL")
px = stock.history(start = "2013-12-01", end = "2024-02-01")

c:\Users\ibast\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\base.py:291: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Dividends"].fillna(0, inplace=True)
c:\Users\ibast\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\base.py:292: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are se

## Parametrize the download

+ Generally, we will look to separate every parameter and setting from functions.
+ If we had a few stocks, we could cycle through them. We need a place to store the list of tickers (a db or file, for example).
+ Store a csv file with a few stock tickers. The location of the file is a setting, the contents of this file are parameters.
+ Use **environment variables** to pass parameters.

In [6]:
ticker_file = os.getenv("TICKERS")
tickers = pd.read_csv(ticker_file).sample(50, random_state=42)

Collecting padas data frames

+ From the [documentation](https://pandas.pydata.org/docs/user_guide/merging.html):

> [`concat()`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html#pandas.concat) makes a full copy of the data, and iteratively reusing `concat()` can create unnecessary copies. Collect all DataFrame or Series objects in a list before using `concat()`.

+ We can string operation togethers using dot operations. Enclose the line in parenthesis and add linebreaks for readability.

In [7]:
# List to hold final results
px_list = list()

for k, row in tickers.iterrows():  # Produces an iterator that returns index and row

    stock = yf.Ticker(row['ticker'])
    print(f'Processing {row["ticker"]}')
    
    px = (stock
          .history(start = pd.to_datetime("2013-12-01"), 
                   end = pd.to_datetime("2024-02-01"))
          .reset_index()   # Reset index to get date as a column
          .assign(ticker = row['ticker']))    # Add ticker
    
    if px.shape[0] == 0:
        print(f'No data for {row["ticker"]}')  # Validate: do not fail silently.
        continue
    print(f'Downloaded {px.shape}.')
    px_list.append(px)
px_dt = pd.concat(px_list, axis = 0)
print(f'Final shape {px_dt.shape}.')

Processing JCI


c:\Users\ibast\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\base.py:291: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Dividends"].fillna(0, inplace=True)
c:\Users\ibast\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\base.py:292: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are se

Downloaded (2558, 9).
Processing BSX


c:\Users\ibast\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\base.py:291: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Dividends"].fillna(0, inplace=True)
c:\Users\ibast\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\base.py:291: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting',

Downloaded (2558, 9).
Processing LIN


...

c:\Users\ibast\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\base.py:291: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Dividends"].fillna(0, inplace=True)
c:\Users\ibast\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\base.py:292: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are se

Downloaded (2558, 9).
Processing ABNB
Downloaded (789, 9).
Processing MDT


c:\Users\ibast\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\base.py:291: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Dividends"].fillna(0, inplace=True)
c:\Users\ibast\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\base.py:292: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are se

Downloaded (2558, 9).
Processing AMZN
Downloaded (2558, 9).
Final shape (118444, 9).


c:\Users\ibast\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\base.py:291: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Dividends"].fillna(0, inplace=True)
c:\Users\ibast\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\base.py:291: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting',

## Reliability

+ Keppelman (2017) defines *reliability* as:

    - A system should continue to work correctly. 
    - To work correctly means performing the correct function at the desired level of performance, even in the face of adversity such as hardware or software faults, and even human error. 

+ *Faults* are things that can go wrong.
+ Sytems that can cope with (certain types of) faults are called *fault-tolerant* or *resilient*.
+ A fault is different than a failure. 
    
    - A *fault* occurs when a component of the system deviates from spec.
    - A *failure*  is when the system stops providing the required service to the user.

+ In our simple example, we handle the fault that occurs when one ticker is not found and log it using *warning*.


# Storing Data in CSV



+ We have some data. How do we store it?
+ We can compare two options, CSV and Parqruet, by measuring their performance:

    - Time to save.
    - Space required.

In [8]:
def get_dir_size(path='.'):
    '''Returns the total size of files contained in path.'''
    total = 0
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_file():
                total += entry.stat().st_size
            elif entry.is_dir():
                total += get_dir_size(entry.path)
    return total

In [9]:
import time

In [10]:
temp = os.getenv("TEMP_DATA")
os.makedirs(temp, exist_ok=True)
stock_path = os.path.join(temp, "stock_px.csv")

In [11]:
start = time.time()
px_dt.to_csv(stock_path, index = False)
end = time.time()

print(f'Writing to dt ({px_dt.shape})csv took {end - start} seconds.')
print(f'Csv file size { os.path.getsize(stock_path)*1e-6 } MB')

Writing to dt ((118444, 9))csv took 1.7239422798156738 seconds.
Csv file size 12.615276999999999 MB


## Save Data to Parquet

### Dask 

We can work with with large data sets and parquet files. In fact, recent versions of pandas support pyarrow data types and future versions will require a pyarrow backend. The pyarrow library is an interface between Python and the Appache Arrow project. The [parquet data format](https://parquet.apache.org/) and [Arrow](https://arrow.apache.org/docs/python/parquet.html) are projects of the Apache Foundation.

However, Dask is much more than an interface to Arrow: Dask provides parallel and distributed computing on pandas-like dataframes. It is also relatively easy to use, bridging a gap between pandas and Spark. 

In [12]:
import dask.dataframe as dd

In [13]:
px_dd = dd.from_pandas(px_dt, npartitions = len(tickers))
parquet_path = os.path.join(temp, "stock_px.parquet")

start = time.time()
px_dd.to_parquet(parquet_path, engine = "pyarrow")
end = time.time()

print(f'Writing dd ({px_dt.shape}) to parquet took {end - start} seconds.')
print(f'Parquet file size { get_dir_size(parquet_path)*1e-6 } MB')

Writing dd ((118444, 9)) to parquet took 0.981921911239624 seconds.
Parquet file size 5.569325999999999 MB


### Parquet files and Dask Dataframes

+ Parquet files are immutable: once written, they cannot be modified.
+ Dask DataFrames are a useful implementation to manipulate data stored in parquets.
+ Parquet and Dask are not the same: parquet is a file format that can be accessed by many applications and programming languages (Python, R, PowerBI, etc.), while Dask is a package in Python to work with large datasets using distributed computation.
+ **Dask is not for everything** (see [Dask DataFrames Best Practices](https://docs.dask.org/en/stable/dataframe-best-practices.html)). 

    - Consider cases suchas small to larrge joins, where the small dataframe fits in memory, but the large one does not. 
    - If possible, use pandas: reduce, then use pandas.
    - Pandas performance tips apply to Dask.
    - Use the index: it is beneficial to have a well-defined index in Dask DataFrames, as it may speed up searching (filtering) the data. A one-dimensional index is allowed.
    - Avoid (or minimize) full-data shuffling: indexing is an expensive operations. 
    - Some joins are more expensive than others. 

        * Not expensive:

            - Join a Dask DataFrame with a pandas DataFrame.
            - Join a Dask DataFrame with another Dask DataFrame of a single partition.
            - Join Dask DataFrames along their indexes.

        * Expensive:

            - Join Dask DataFrames along columns that are not their index.


# How do we store prices?

+ We can store our data as a single blob. This can be difficult to maintain, especially because parquet files are immutable.
+ Strategy: organize data files by ticker and date. Update only latest month.



In [14]:
# CLean up before start
PRICE_DATA = os.getenv("PRICE_DATA")
import shutil
if os.path.exists(PRICE_DATA):
    shutil.rmtree(PRICE_DATA)

In [15]:
for ticker in px_dt.ticker.unique():
    ticker_dt = px_dt[px_dt.ticker == ticker]
    ticker_dt = ticker_dt.assign(year = ticker_dt.Date.dt.year)
    for yr in ticker_dt.year.unique():
        yr_dt = ticker_dt[ticker_dt.year == yr]
        yr_path = os.path.join(PRICE_DATA, ticker, f"{ticker}_{yr}.parquet")
        os.makedirs(os.path.dirname(yr_path), exist_ok=True)
        yr_dt.to_parquet(yr_path, engine = "pyarrow")
    

Why would we want to store data this way?

+ Easier to maintain. We do not update old data, only recent data.
+ We can also access all files as follows.

# Load, Transform and Save 

## Load

+ Parquet files can be read individually or as a collection.
+ `dd.read_parquet()` can take a list (collection) of files as input.
+ Use `glob` to get the collection of files.

In [16]:
from glob import glob

parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

## Transform

+ This transformation step will create a *Features* data set. In our case, features will be stock returns (we obtained prices).
+ Dask dataframes work like pandas dataframes: in particular, we can perform groupby and apply operations.
+ Notice the use of [an anonymous (lambda) function](https://realpython.com/python-lambda/) in the apply statement.

In [17]:
import numpy as np
dd_rets = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    positive_return = lambda x: (x['returns'] > 0)*1
))

C:\Users\ibast\AppData\Local\Temp\ipykernel_29216\3091835559.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_rets = (dd_px.groupby('ticker', group_keys=False).apply(


## Lazy Exection

What does `dd_rets` contain?

In [18]:
dd_rets

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Close_lag_1,returns,positive_return
npartitions=49,,,,,,,,,,,,
A,datetime64[ns],float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,int64
ABNB,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,...,...,...,...,...,...,...,...,...,...,...,...
YUM,...,...,...,...,...,...,...,...,...,...,...,...


+ Dask is a lazy execution framework: commands will not execute until they are required. 
+ To trigger an execution in dask use `.compute()`.

In [19]:
dd_rets.compute()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Close_lag_1,returns,positive_return
ticker,,,,,,,,,,,,
A,2024-01-02,137.491417,139.879279,137.212834,138.048584,1441600.0,0.0,0.0,2024,NaN,NaN,0
A,2024-01-03,137.302382,137.302382,130.407422,130.496964,2074500.0,0.0,0.0,2024,138.048584,-0.054703,0
A,2024-01-04,129.890062,130.835256,129.531881,130.337784,2446600.0,0.0,0.0,2024,130.496964,-0.001220,0
A,2024-01-05,129.342812,131.292910,127.969783,129.899979,1394000.0,0.0,0.0,2024,130.337784,-0.003359,0
A,2024-01-08,129.482108,132.894776,129.153774,132.705734,1311400.0,0.0,0.0,2024,129.899979,0.021599,1
...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,2024-01-25,127.913548,128.386458,126.642582,128.297791,1871700.0,0.0,0.0,2024,127.529289,0.006026,1
YUM,2024-01-26,128.002206,128.760844,126.770657,127.184456,1157000.0,0.0,0.0,2024,128.297791,-0.008678,0
YUM,2024-01-29,127.194317,128.622910,126.396261,128.622910,1482100.0,0.0,0.0,2024,127.184456,0.011310,1


## Save

+ Apply transformations to calculate daily returns
+ Store the enriched data, the silver dataset, in a new directory.
+ Should we keep the same namespace? All columns?

In [20]:
# CLean up before start
FEATURES_DATA = os.getenv("FEATURES_DATA")
if os.path.exists(FEATURES_DATA):
    shutil.rmtree(FEATURES_DATA)

In [21]:
FEATURES_DATA = os.getenv("FEATURES_DATA")
dd_rets.to_parquet(FEATURES_DATA, engine = "pyarrow")

# A few notes

# Jupyter? 

+ We have drafted our code in a Jupyter Notebook. 
+ Finalized code should be written in Python modules.

## Object oriented programming?

+ We can use classes to keep parameters and functions together.
+ We *could* use Object Oriented Programming, but parallelization of data manipulation and modelling tasks benefits from *Functional Programming*.
+ An Idea: 

    - [Data Oriented Programming](https://blog.klipse.tech/dop/2022/06/22/principles-of-dop.html).
    - Use the class to bundle together parameters and functions.
    - Use stateless operations and treat all data objects as immutable (we do not modify them, we overwrite them).
    - Take advantage of [`@staticmethod`](https://realpython.com/instance-class-and-static-methods-demystified/).

The code is in `./05_src/`.

Our original design was:

![](./images/02_target_pipeline_manager.png)



Below, we demonstrate how our class implementation works.

Before we begin, clean the price directory (we will download everything again).

In [22]:
# CLean up before start

if os.path.exists(PRICE_DATA):
    shutil.rmtree(PRICE_DATA)
if os.path.exists(FEATURES_DATA):
    shutil.rmtree(FEATURES_DATA)


Now, instantiate a DataManager object and download all the data.

In [23]:
from data_manager import DataManager
dm = DataManager()

In [24]:

dm.download_all()

2024-10-18 14:33:55,488, data_manager.py, 43, INFO, Getting price data for all tickers.
2024-10-18 14:33:55,488, data_manager.py, 43, INFO, Getting price data for all tickers.
2024-10-18 14:33:55,492, data_manager.py, 52, INFO, Getting tickers from ../../05_src/data/tickers/sp500_wiki.csv
2024-10-18 14:33:55,492, data_manager.py, 52, INFO, Getting tickers from ../../05_src/data/tickers/sp500_wiki.csv
2024-10-18 14:33:55,516, data_manager.py, 58, INFO, Processing all tickers
2024-10-18 14:33:55,516, data_manager.py, 58, INFO, Processing all tickers
2024-10-18 14:33:55,528, data_manager.py, 73, INFO, Processing ticker MMM
2024-10-18 14:33:55,528, data_manager.py, 73, INFO, Processing ticker MMM
2024-10-18 14:33:55,534, data_manager.py, 108, INFO, Getting stock price data for MMM from 2000-01-01 to 2024-10-18
2024-10-18 14:33:55,534, data_manager.py, 108, INFO, Getting stock price data for MMM from 2000-01-01 to 2024-10-18


[*********************100%***********************]  1 of 1 completed

c:\Users\ibast\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\base.py:291: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Dividends"].fillna(0, inplace=True)
c:\Users\ibast\AppData\Local\Programs\Python\Python310\lib\site-packages\yfinance\base.py:292: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are se

2024-10-18 14:33:57,695, data_manager.py, 73, INFO, Processing ticker AOS
2024-10-18 14:33:57,695, data_manager.py, 73, INFO, Processing ticker AOS
2024-10-18 14:33:57,697, data_manager.py, 108, INFO, Getting stock price data for AOS from 2000-01-01 to 2024-10-18
2024-10-18 14:33:57,697, data_manager.py, 108, INFO, Getting stock price data for AOS from 2000-01-01 to 2024-10-18


[*********************100%***********************]  1 of 1 completed

...



1 Failed download:
- ZION: No data found for this date range, symbol may be delisted


2024-10-18 21:45:19,060, data_manager.py, 100, WARNING, No data found for ZION
2024-10-18 21:45:19,060, data_manager.py, 100, WARNING, No data found for ZION
2024-10-18 21:45:19,064, data_manager.py, 73, INFO, Processing ticker ZTS
2024-10-18 21:45:19,064, data_manager.py, 73, INFO, Processing ticker ZTS
2024-10-18 21:45:19,067, data_manager.py, 108, INFO, Getting stock price data for ZTS from 2000-01-01 to 2024-10-18
2024-10-18 21:45:19,067, data_manager.py, 108, INFO, Getting stock price data for ZTS from 2000-01-01 to 2024-10-18


[*********************100%***********************]  1 of 1 completed

2024-10-18 21:45:19,089, data_manager.py, 82, INFO, Saving data for ZTS by year
2024-10-18 21:45:19,089, data_manager.py, 82, INFO, Saving data for ZTS by year
2024-10-18 21:45:19,093, data_manager.py, 100, WARNING, No data found for ZTS
2024-10-18 21:45:19,093, data_manager.py, 100, WARNING, No data found for ZTS




1 Failed download:
- ZTS: No data found for this date range, symbol may be delisted


Finally, add features to our data set and save to a *feature store*.

In [26]:
dm.featurize()

2024-10-19 09:09:37,618, data_manager.py, 117, INFO, Creating features data.
2024-10-19 09:09:37,618, data_manager.py, 117, INFO, Creating features data.
2024-10-19 09:09:37,658, data_manager.py, 128, INFO, Loading price data from ../../05_src/data/prices/
2024-10-19 09:09:37,658, data_manager.py, 128, INFO, Loading price data from ../../05_src/data/prices/
2024-10-19 09:09:43,207, data_manager.py, 136, INFO, Creating features
2024-10-19 09:09:43,207, data_manager.py, 136, INFO, Creating features
c:\Users\ibast\DSI_Production\01_materials\labs\../../05_src\data_manager.py:140: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  .apply(
2024-10-19 09:09:43,225, data_manager.py, 154, INFO, Creating target
2024-10-19 09:09:43,225, data_manager.py, 154, INFO, C